In [1]:
import pandas as pd
import numpy as np
import os
#os.chdir("../")

In [2]:
os.getcwd()

'c:\\Users\\wpiel\\web_dev\\wucziapping\\backend\\question_generator'

In [13]:
possible_ans = pd.read_csv(r"outputs/build_question_related_models/question_model.csv")

In [14]:
possible_ans

,question_key,is_hard,question_category_id,correct_answer_id,possible_answers_id,question_text_id
0,1,1,1,1,1,1
1,2,1,1,2,2,1
2,3,1,1,3,3,1
3,4,1,1,1,4,1
4,5,1,1,4,5,1
...,...,...,...,...,...,...
6287,6288,1,7,84,6288,74
6288,6289,1,7,84,6289,74
6289,6290,1,7,84,6290,75
6290,6291,1,7,84,6291,75


In [10]:
possible_ans["is_hard"] = possible_ans["is_hard"].replace({"PRAWDA":1, "FAŁSZ":0})

In [11]:
possible_ans.columns
possible_ans=possible_ans[["question_key","is_hard","correct_answer_id", "possible_answers_id", "question_category_id", "question_text_id"]]

In [12]:
possible_ans.to_csv(r"outputs/build_question_related_models/question_model.csv", index=False)

In [2]:
no_precambr_data = pd.read_excel(f"{os.getcwd()}\inputs\wucziapping_data.xlsx", sheet_name="reszta")
precambr_data = pd.read_excel(f"{os.getcwd()}\inputs\wucziapping_data.xlsx", sheet_name="Prekambr")

In [12]:
data = no_precambr_data
dataPrekambr = precambr_data
isPrekambr = True
isHard = False
startColIdx = 1
endColIdx = 3
multiplyRowsIdx = 2
colName1 = "Eon"
colName2 = "System"
questionPattern = "Prekambr - wybierz system ery"

if isPrekambr:
    df = dataPrekambr.iloc[:, [startColIdx,endColIdx]]
else:
    df = data.iloc[:, [startColIdx,endColIdx]]

df = df.loc[df.index.repeat(multiplyRowsIdx)]

random_answers = []
questions = []

if isHard:
    scope = np.unique(
            np.concatenate((data["PIETRO"].unique(), data["ODDZIAL"].unique(), data["SYSTEM"].unique(), data["ERA"].unique(), 
                            dataPrekambr["Jednostka nieformalna"].unique(), dataPrekambr["Eon"].unique(), 
                            dataPrekambr["Era"].unique(), dataPrekambr["System"].unique()))
                            )

else:
    if isPrekambr:
        scope = dataPrekambr[colName2].unique()
    else:
        scope = data[colName2].unique()

print(scope)

for i in df[colName1]:
    
    if isPrekambr:
        temp_correct = dataPrekambr[dataPrekambr[colName1]==i][colName2].array
    else:
        temp_correct = data[data[colName1]==i][colName2].array

    indexes = np.nonzero(np.in1d(scope, temp_correct))[0]

    print(temp_correct)

    temp_scope = np.delete(scope, indexes)
    
    print(temp_scope)

    if isHard:
        temp = np.concatenate((np.random.choice(temp_scope,4), np.random.choice(temp_correct,1)))
    else:
        temp = np.concatenate((np.random.choice(temp_scope,3), np.random.choice(temp_correct,1)))

    np.random.shuffle(temp)

    if not isHard:
        temp = np.append(temp, np.nan)

    random_answers.append(temp)
    questions.append(f"{questionPattern} {i}")

p1 = pd.DataFrame(questions, columns=["question"])
p2 = pd.DataFrame(random_answers, columns=["A", "B", "C", "D", "E"]).drop_duplicates()
p3 = pd.concat([p1, p2], axis=1)

all_correct_answers = []

for index, row in p3.iterrows():
    temp_val1 = row["question"].split(" ")[-1]

    if temp_val1.isdigit() or temp_val1 in ("górna", "dolna", "środkowa", "górny", "środkowy", "dolny"):
        temp_val1 = row["question"].split(" ")[-2:]
        temp_val1 = " ".join(temp_val1)

    temp_abcde = row[["A", "B", "C", "D", "E"]].tolist()

    if isPrekambr:
        correct_answers = dataPrekambr[dataPrekambr[colName1]==temp_val1][colName2].tolist()
    else:
        correct_answers = data[data[colName1]==temp_val1][colName2].tolist()

    for answer in temp_abcde:
        if answer in correct_answers:
            all_correct_answers.append(answer)
            break

p4 = pd.concat([p3, pd.DataFrame(all_correct_answers, columns=["correct_answer"])], axis=1)

['ediakar' 'kriogen' 'ton' 'sten' 'ektas' 'kalim' 'stater' 'orosir' 'riak'
 'sider' 'brak']
<NumpyExtensionArray>
['ediakar', 'kriogen',     'ton',    'sten',   'ektas',   'kalim',  'stater',
  'orosir',    'riak',   'sider']
Length: 10, dtype: object
['brak']
<NumpyExtensionArray>
['ediakar', 'kriogen',     'ton',    'sten',   'ektas',   'kalim',  'stater',
  'orosir',    'riak',   'sider']
Length: 10, dtype: object
['brak']
<NumpyExtensionArray>
['ediakar', 'kriogen',     'ton',    'sten',   'ektas',   'kalim',  'stater',
  'orosir',    'riak',   'sider']
Length: 10, dtype: object
['brak']
<NumpyExtensionArray>
['ediakar', 'kriogen',     'ton',    'sten',   'ektas',   'kalim',  'stater',
  'orosir',    'riak',   'sider']
Length: 10, dtype: object
['brak']
<NumpyExtensionArray>
['ediakar', 'kriogen',     'ton',    'sten',   'ektas',   'kalim',  'stater',
  'orosir',    'riak',   'sider']
Length: 10, dtype: object
['brak']
<NumpyExtensionArray>
['ediakar', 'kriogen',     'ton',    'ste

In [6]:
p4

,question,A,B,C,D,E,correct_answer
0,Prekambr - wybierz system ery proterozoik,brak,brak,brak,orosir,NaN,orosir
1,Prekambr - wybierz system ery proterozoik,brak,brak,ton,brak,NaN,ton
2,Prekambr - wybierz system ery proterozoik,brak,brak,riak,brak,NaN,riak
3,Prekambr - wybierz system ery proterozoik,brak,brak,orosir,brak,NaN,orosir
4,Prekambr - wybierz system ery proterozoik,brak,brak,brak,sten,NaN,sten
5,Prekambr - wybierz system ery proterozoik,brak,brak,brak,ektas,NaN,ektas
6,Prekambr - wybierz system ery proterozoik,brak,brak,ektas,brak,NaN,ektas
7,Prekambr - wybierz system ery proterozoik,NaN,NaN,NaN,NaN,NaN,stater
8,Prekambr - wybierz system ery proterozoik,stater,brak,brak,brak,NaN,ektas
9,Prekambr - wybierz system ery proterozoik,ektas,brak,brak,brak,NaN,sider
